# Überblick annotierter Bilder

In [17]:
# Import Packages
from Code.find_path_nextcloud import find_path_nextcloud
import matplotlib.pyplot as plt
import spectral as spy
import pandas as pd
import numpy as np
import os

In [52]:
def isfloat(num):
    try:
        float(num)
        return True
    except ValueError:
        return False

In [57]:
def import_labeled_data():

    # Pfad Nextcloud bestimmen
    path_nextcloud = find_path_nextcloud()

    # Festlegung, welches Grid zusammengelegt werden soll
    windowsize_r = 200
    windowsize_c = 200

    # Bestimmung annotation_folder
    grid_folder = path_nextcloud + "Daten_Gyrocopter/Oldenburg/Teilbilder/grid_" + str(windowsize_r) + "_" + str(windowsize_c)
    labeled_folder = grid_folder + '/labeled/'

    # Liste aller Dateien in annotation_folder erstellen
    files = os.listdir(labeled_folder)

    # Aus Liste files .hdr Dateien löschen
    for file in files:
        if not file.endswith('.dat'):
            files.remove(file)

    # Spaltennamen des DataFrames bilden
    path_dat = labeled_folder + os.path.splitext(files[0])[0] + '.dat'
    path_hdr = labeled_folder + os.path.splitext(files[0])[0] + '.hdr'

    # load image
    img = spy.envi.open(file=path_hdr, image=path_dat)

    # convert only wavelength into dataframe and round numbers
    value_bands = ['hsi_band_' + str(int(float(x))) + '_nm' for x in img.metadata['wavelength'] if isfloat(x)]
    value_bands.extend(['thermal', 'dom'])

    label_bands = ['None', 'Wiese', 'Straße', 'Auto', 'See', 'Schienen', 'Haus', 'Wald', 'label']

    bands = []
    bands.extend(value_bands)
    bands.extend(label_bands)
    bands.append('picture_name')

    df_annotations = pd.DataFrame(columns=bands)

    # labeled Bilder erstellen
    for filename in files[:-1]:
        path_dat = labeled_folder + os.path.splitext(filename)[0] + '.dat'
        path_hdr = labeled_folder + os.path.splitext(filename)[0] + '.hdr'
        img = spy.envi.open(file=path_hdr, image=path_dat)

        arr = img.load()

        df = pd.DataFrame(arr.reshape((40000, (arr.shape[2]))), columns=bands[:-1])
        df['picture_name'] = os.path.splitext(filename)[0]

        df_annotations = pd.concat([df_annotations, df], ignore_index=True)

    return df_annotations


In [58]:
df_annotations = import_labeled_data()

In [59]:
df_annotations.shape

(560000, 119)

In [60]:
df_annotations['picture_name'].value_counts()

Teilbild_Oldenburg_00000000_00000000_0_0_          40000
Teilbild_Oldenburg_00000000_00000002_0_400_        40000
Teilbild_Oldenburg_00000000_00000008_0_1600_       40000
Teilbild_Oldenburg_00000000_00000009_0_1800_       40000
Teilbild_Oldenburg_00000000_00000014_0_2800_       40000
Teilbild_Oldenburg_00000000_00000015_0_3000_       40000
Teilbild_Oldenburg_00000001_00000014_200_2800_     40000
Teilbild_Oldenburg_00000001_00000016_200_3200_     40000
Teilbild_Oldenburg_00000002_00000002_400_400_      40000
Teilbild_Oldenburg_00000002_00000008_400_1600_     40000
Teilbild_Oldenburg_00000006_00000010_1200_2000_    40000
Teilbild_Oldenburg_00000007_00000008_1400_1600_    40000
Teilbild_Oldenburg_00000008_00000005_1600_1000_    40000
Teilbild_Oldenburg_00000008_00000010_1600_2000_    40000
Name: picture_name, dtype: int64

In [61]:
df_objects = df_annotations.loc[df_annotations['label'] != 0, ]
df_objects.shape

(367012, 119)

In [62]:
df_objects.shape[0] / df_annotations.shape[0]

0.6553785714285715

In [63]:
df_objects['picture_name'].value_counts()

Teilbild_Oldenburg_00000008_00000010_1600_2000_    37119
Teilbild_Oldenburg_00000008_00000005_1600_1000_    35665
Teilbild_Oldenburg_00000000_00000009_0_1800_       33919
Teilbild_Oldenburg_00000000_00000002_0_400_        32886
Teilbild_Oldenburg_00000007_00000008_1400_1600_    31528
Teilbild_Oldenburg_00000002_00000008_400_1600_     31054
Teilbild_Oldenburg_00000000_00000000_0_0_          30655
Teilbild_Oldenburg_00000000_00000014_0_2800_       23342
Teilbild_Oldenburg_00000000_00000015_0_3000_       22489
Teilbild_Oldenburg_00000001_00000014_200_2800_     21210
Teilbild_Oldenburg_00000006_00000010_1200_2000_    20796
Teilbild_Oldenburg_00000002_00000002_400_400_      20546
Teilbild_Oldenburg_00000000_00000008_0_1600_       20323
Teilbild_Oldenburg_00000001_00000016_200_3200_      5480
Name: picture_name, dtype: int64

In [65]:
w = df_objects.groupby(['label'], as_index=False).size()

In [66]:
class_objects = {0:'None', 1: 'Wiese', 2: 'Straße', 3: 'Auto', 4: 'See', 5: 'Schienen', 6: 'Haus', 7: 'Wald'}

In [68]:
w['label'] = w['label'].replace(class_objects)

In [69]:
w

,label,size
0,Wiese,111052
1,Auto,10747
2,See,14305
3,Schienen,4634
4,Haus,113564
5,Wald,112710
